In [1]:
from LandBase import *

def adm_law_to_db(con) :
    ####
    # 행정동-법정동 매핑정보를 DB에 입력 (출처: 행정표준코드 관리시스템, 행정안전부 -> 주민등록주소코드 변경내역)
    # 테이블: adm_law_map
    # 법정동 코드 (10자리) : 실제주소. 시도(2)+시군구(3)+읍면동(3)+리(2)
    ####
    df = pd.read_csv('./source_files/adm_law_mapping.csv', sep=',', dtype={'행정동코드':str, '법정동코드':str}, encoding='utf-8')
    df = df[ (df['시도명'].str.slice(0, 2).str.contains('서울|경기|인천')) ]
    df.rename(columns={'행정동코드':'admd_cd', '시도명':'sido_nm', '시군구명':'sgg_nm', '읍면동명':'admd_nm', \
                       '법정동코드':'lawd_cd', '동리명':'lawd_nm', '생성일자':'from_dt', '말소일자':'to_dt'}, inplace=True)
    # DB 처리
    table_name = 'adm_law_map'
    sql = 'delete from ' + table_name
    con.execute(sql)
    df.to_sql(name=table_name, con=con,  if_exists='append', index=False, chunksize=1000)
    sql = 'delete from ' + table_name + ' where to_dt < \'19970101\''
    con.execute(sql)
    sql = 'update ' + table_name \
    + ' set lawd_nm = sgg_nm where replace(sgg_nm, \' \', \'\') = replace(lawd_nm, \' \', \'\') ' \
    + ' and char_length(sgg_nm) != char_length(lawd_nm)'
    con.execute(sql)
    
def lawd_to_db(con) :
    ####
    # 법정동 정보를 DB에 입력 (출처: 행정표준코드 관리시스템)
    # 테이블: lawd_mstr
    ####
    df = pd.read_csv('./source_files/lawd_all_data.txt', sep='\t', dtype={'법정동코드':str}, encoding='utf-8')
    df = df[ (df['법정동명'].str.slice(0, 2).str.contains('서울|경기|인천')) ]
    df.rename(columns={'법정동코드':'lawd_cd', '법정동명':'lawd_nm', '폐지여부':'abolished_yn'}, inplace=True)
    # DB 처리
    table_name = 'lawd_mstr'
    delete_sql = 'delete from ' + table_name
    con.execute(delete_sql)
    df.to_sql(name=table_name, con=con,  if_exists='append', index=False, chunksize=1000)

def admd_to_db (engine) :
    ####
    # 행정동 정보를 DB에 입력 (출처: )
    # 테이블: admd_mstr (행정=administrative)
    ####
    df = pd.read_excel('http://sgis.kostat.go.kr/OpenAPI2/upload/unitarea_code.xls', skiprows=1)
    df.rename(columns={'시도코드':'sido_cd', '시도명칭':'sido_nm', \
                       '시군구코드':'sgg_cd', '시군구명칭':'sgg_nm', \
                       '읍면동코드':'admd_cd','읍면동명칭':'admd_nm'}, inplace=True)
    # DB 처리
    table_name = 'admd_mstr'
    delete_sql = 'delete from ' + table_name
    con.execute(delete_sql)
    df.to_sql(name=table_name, con=con,  if_exists='append', index=False, chunksize=1000)


In [4]:
# DB Connection Open
con = get_db_connection()

# 법정동 정보 입력
# 테이블: lawd_mstr 
#lawd_to_db(con)

# 행정동 정보 입력
# 테이블: admd_mstr 
#admd_to_db(con)

# 법정동-행정동 매핑 정보 입력
# 테이블: adm_law_map
adm_law_to_db(con)

# DB Connection Close
disconnect_db(con)

print('Program Finised')

Program Finised
